In [5]:
%use dataframe
%use kandy
%use ktor-client

import io.ktor.client.*
import io.ktor.client.engine.cio.*
import io.ktor.client.statement.*
import io.ktor.client.request.*
import io.ktor.http.*
import kotlinx.coroutines.runBlocking
import kotlinx.serialization.*
import kotlinx.serialization.json.*
import java.time.*
import java.time.format.DateTimeFormatter
import org.jetbrains.kotlinx.dataframe.api.*



In [6]:
// --- 1) Data classes & config ---
@Serializable
data class FrankfurterResponse(
    val base: String,
    val date: String,
    val rates: Map<String, Double>
)
data class MonthRate(val month: String, val rate: Double, val pctChange: Double?)

// We no longer need an API key:
val baseUrl        = "https://api.frankfurter.app"
val targetCurrency = "EUR"
val client         = HttpClient(CIO)
val formatter      = DateTimeFormatter.ofPattern("MMM yyyy")

// --- 2) Build the 1st-of-month dates for the last 12 months ---
val today    = LocalDate.now()
val firstDay = today.withDayOfMonth(1)
val dates12  = (0 until 12)
    .map { firstDay.minusMonths(it.toLong()) }
    .reversed()

// --- 3) Fetch rates & compute % change in one go ---
val ratesWithPct = runBlocking {
    val raw = dates12.map { date ->
        val isoDate = date.format(DateTimeFormatter.ISO_DATE)
        val url     = "$baseUrl/$isoDate?from=USD&to=$targetCurrency"
        val resp    = client.get(url)
        require(resp.status.value == 200) { "HTTP ${resp.status}" }
        val body    = resp.bodyAsText()

        // <-- NEW: configure parser to ignore unknown "amount" key
        val parser  = Json { ignoreUnknownKeys = true }
        val json    = parser.decodeFromString<FrankfurterResponse>(body)

        MonthRate(date.format(formatter), json.rates[targetCurrency]!!, null)
    }
    raw.mapIndexed { i, mr ->
        val pct = if (i == 0) null else (mr.rate - raw[i-1].rate) / raw[i-1].rate * 100
        MonthRate(mr.month, mr.rate, pct)
    }
}


In [7]:
// 1) Turn your list into a proper two-column DataFrame:
val df = ratesWithPct
    .toDataFrame()                                 // creates columns "month","rate","pctChange"
    .select("month","pctChange")                   // drop the raw rate if you like
    .rename("month" to "Month",                     // make them pretty
        "pctChange" to "PctChange")

df

Month,PctChange
Jan 2025,null
Feb 2025,"-0,038439"
Mar 2025,"-0,173562"
Apr 2025,"-3,493941"
May 2025,"-5,143696"
Jun 2025,"0,299108"
Jul 2025,"-3,987935"
Aug 2025,"3,560715"
Sept 2025,"-2,654837"
Oct 2025,"-0,077319"


In [8]:
// 1) Build a true two-column DataFrame of Month & Rate
val dfValues = ratesWithPct
    .toDataFrame()                                 // gives you columns "month","rate","pctChange"
    .select("month","rate")                        // drop pctChange
    .rename("month" to "Month",                    // pretty up the headers
        "rate"  to "Rate")

dfValues.print()  // confirm your table

// 2) Plot Month vs. Rate
dfValues.plot {
    line {
        x("Month")
        y("Rate")
    }
    layout {
        title = "USD→EUR Exchange Rate, Last 12 Months"
        size  = 800 to 400
    }
    y {
        axis {
            name = "Exchange Rate (EUR per USD)"
        }
    }
}

        Month    Rate
  0  Jan 2025 0,96256
  1  Feb 2025 0,96219
  2  Mar 2025 0,96052
  3  Apr 2025 0,92696
  4  May 2025 0,87928
  5  Jun 2025 0,88191
  6  Jul 2025 0,84674
  7  Aug 2025 0,87689
  8 Sept 2025 0,85361
  9  Oct 2025 0,85295
 10  Nov 2025 0,86550
 11  Dec 2025 0,85866



<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; padding: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="6Eaedj"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 const forceImmediateRender = false;
 const responsive = false;
 
 let sizing = {
 width_mode: "FIXED",
 height_mode: "FIXED",
 width: 800.0, 
 height: 400.0 
 };
 
 const preferredWidth = document.body.dataset.letsPlotPreferredWidth;
 if (preferredWidth !== undefined) {
 sizing = {
 width_mode: 'FIXED',
 height_mode: 'SCALED',
 width: parseFloat(preferredWidth)
 };
 }
 
 const containerDiv = document.getElementById("6Eaedj");
 let fig = null;
 
 function renderPlot() {
 if (fig === null) {
 const plotSpec = {
"ggtitle":{
"text":"USD→EUR Exchange Rate, Last 12 Months"
},
"mapping":{
},
"data":{
"Month":["Jan 2025","Feb 2025","Mar 2025","Apr 2025","May 2025","Jun 2025","Jul 2025","Aug 2025","Sept 2025","Oct 2025","Nov 2025","Dec 2025"],
"Rate":[0.96256,0.96219,0.96052,0.92696,0.87928,0.88191,0.84674,0.87689,0.85361,0.85295,0.8655,0.85866]
},
"ggsize":{
"width":800.0,
"height":400.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Exchange Rate (EUR per USD)",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"Month",
"y":"Rate"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"Month"
},{
"type":"float",
"column":"Rate"
}]
},
"spec_id":"5"
};
 fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, containerDiv, sizing);
 } else {
 fig.updateView({});
 }
 }
 
 const renderImmediately = 
 forceImmediateRender || (
 sizing.width_mode === 'FIXED' && 
 (sizing.height_mode === 'FIXED' || sizing.height_mode === 'SCALED')
 );
 
 if (renderImmediately) {
 renderPlot();
 }
 
 if (!renderImmediately || responsive) {
 // Set up observer for initial sizing or continuous monitoring
 var observer = new ResizeObserver(function(entries) {
 for (let entry of entries) {
 if (entry.contentBoxSize && 
 entry.contentBoxSize[0].inlineSize > 0) {
 if (!responsive && observer) {
 observer.disconnect();
 observer = null;
 }
 renderPlot();
 if (!responsive) {
 break;
 }
 }
 }
 });
 
 observer.observe(containerDiv);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M13.01727532147808 12.869173854793644 L13.01727532147808 12.869173854793644 L78.10365192886847 13.691414284127859 L143.19002853625886 17.402607573284513 L208.27640514364927 91.98203678531672 L273.3627817510396 197.93993859788247 L338.44915835843 192.09536473531853 L403.53553496582043 270.25265095066925 L468.6219115732108 203.25116731709454 L533.7082881806012 254.98564622222307 L598.7946647879917 256.45234536644057 L663.881041395382 228.56283891200042 L728.9674180027724 243.76317549752798 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 2025 
 
 
 
 
 
 
 
 
 Feb 2025 
 
 
 
 
 
 
 
 
 Mar 2025 
 
 
 
 
 
 
 
 
 Apr 2025 
 
 
 
 
 
 
 
 
 May 2025 
 
 
 
 
 
 
 
 
 Jun 2025 
 
 
 
 
 
 
 
 
 Jul 2025 
 
 
 
 
 
 
 
 
 Aug 2025 
 
 
 
 
 
 
 
 
 Sept 2025 
 
 
 
 
 
 
 
 
 Oct 2025 
 
 
 
 
 
 
 
 
 Nov 2025 
 
 
 
 
 
 
 
 
 Dec 2025 
 
 
 
 
 
 
 
 
 
 
 0.86 
 
 
 
 
 
 
 0.88 
 
 
 
 
 
 
 0.9 
 
 
 
 
 
 
 0.92 
 
 
 
 
 
 
 0.94 
 
 
 
 
 
 
 0.96 
 
 
 
 
 
 
 
 
 USD→EUR Exchange Rate, Last 12 Months 
 
 
 
 
 Exchange Rate (EUR per USD) 
 
 
 
 
 Month